### 복습

In [ ]:
crawling + scrapping
: <a>(link) : 어느 영역(내가 찾는 데이터 - 속성, 구조(부모, 조상))
: focused crawling(전략-내가 원하는 형태의 결과만 받아올 수 있게끔, depth, domain, 영역(html tag, attribute) 제한)
    
# <a> 태그는 Link Page, 즉 일반적으로 페이지 연결을 위해 사용되며 연결 대상이 되는 페이지는 <a> 태그의 href 속성을 통해 정의.

In [ ]:
# 받아오는 결과
: response.headers['content-type'] -> text/html, image/*

In [ ]:
# 그러나 우리는 지금까지 잘 만들어진 사이트만 다님(구글, 네이버, 다음)
# 오늘은 다이나믹 HTML 을 보자

### 수업

In [ ]:
DHTML(AJAX - XHR) ; dynamic html
# Ajax(Asynchronous JavaScript and XML)는 웹 페이지에서 '비동기적'으로 데이터를 요청하고 응답을 받기 위해 사용되는 기술들의 조합
# 이를 통해 브라우저에서는 비동기적으로 데이터의 요청과 응답이 가능하며 동적인 웹을 만들기 위해 사용됨.
# 동기방식으로 진행한다면 각 작업이 순서대로 진행되며 이전의 작업이 끝나야만 다음 작업을 수행함
# 반면 비동기방식은 여러 작업들을 동시다발적으로 실행시키기 때문에 여러 개의 작업을 거의 동시에 진행.
# 동적 웹은 HTML로 전체적인 부분을 만들어놓고 필요에 따라 JS로 DOM을 조작하여 사용자의 정보나 알맞는 데이터가 표시되야 하는
# 일부분만 수정하면 되기 때문에 정적 웹에 비해서 훨씬 적은 데이터 비용으로 유동적인 웹을 보여줄 수 있다.
# Web API중 하나인 이 XHR 객체는 비동기적으로 서버와 통신할 수 있도록 해주는 Ajax의 핵심 역할을 한다.

In [3]:
from urllib.robotparser import RobotFileParser
from requests import get
from requests.compat import urlparse, urljoin
from requests.exceptions import HTTPError
from time import sleep
from bs4 import BeautifulSoup
import re

In [4]:
headers = {
    'user-agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/110.0.0.0 Safari/537.36'
}

#### DHTML

In [ ]:
# 다이나믹 HTML은 요청하면 받는 것은 같음.
# 이 상태에서 또 다른 요청을 AJAX로 함. 그러면 서버가 데이터(XHR)로 응답을 함.
# html 문서를 받은 것이 아니기 때문에 돔을 그리지 않고, 기존 돔(첫 응답)에 끼워맞춤.
# 가져오는 우리 입장에서는 분명히 화면에 그려져 있음. 그런데 돔은 옛날 것만 그리고 있음. 새롭게 받은 것은 반영 안되어 있음.
# 그 대표적 사례가 웹툰

# Ajax는 웹 페이지 전체를 다시 로딩하지 않고도, 웹 페이지의 일부분만을 갱신할 수 있음. (동적)
# 즉 Ajax를 이용하면 백그라운드 영역에서 서버와 통신하여, 그 결과를 웹 페이지의 일부분에만 표시할 수 있는 것

In [5]:
from requests import get
resp = get('https://pythonscraping.com/pages/javascript/ajaxDemo.html')
# 스텝 1
# DOM 은 이 때 그린 것

In [6]:
print(resp.text)

<html>
<head>
<title>Some JavaScript-loaded content</title>
<script src="../js/jquery-2.1.1.min.js"></script>

</head>
<body>
<div id="content">
This is some content that will appear on the page while it's loading. You don't care about scraping this.
</div>

<script>
$.ajax({
    type: "GET",
    url: "loadedContent.php",
    success: function(response){

	setTimeout(function() {
	    $('#content').html(response);
	}, 2000);
    }
  });

function ajax_delay(str){
 setTimeout("str",2000);
}
</script>
</body>
</html>


In [ ]:
# 코드로 접근했을 때랑 실제 브라우저로 접근해서 돔을 열었을 때랑 div id = content 부분이 다름
# ajax로 뒷단에서 비동기 통신을 함
# setTimeout(function() {
# 	    $('#content').html(response);        => 여기서 이 response 가 기존 돔의 id content 내용을 바꿈
# 	}, 2000);
#     }
# 문서를 받아서 돔을 그려내고(처음 요청/응답) 다음 요청 때 뒷단에서 데이터만 받아서 특정 돔에 있는 요소에 받아온 데이터를 끼워넣는 형태
# 기존의 html 보다 동적으로 html 이 변할 수 있음
# 이런 페이지들이 있다는 것을 알고 있을 것. 요새 웹은 이런 식을 많이 씀

In [ ]:
# 브라우저 개발자 도구에서 히스토리를 따라가보기 doc -> js -> xhr

In [ ]:
# <script>
# $.ajax({
#     type: "GET",
#     url: "loadedContent.php",
#     success: function(response){

# 	setTimeout(function() {
# 	    $('#content').html(response);
# 	}, 2000);
#     }
#   });
# 우리는 이걸 해석해야 함. 자바스크립트.

In [7]:
resp = get('https://pythonscraping.com/pages/javascript/loadedContent.php')
resp.text
# 스텝 2
# 데이터 받아온 것

'Here is some important text you want to retrieve! <p/><button id="loadedButton">A button to click!</button>'

In [ ]:
# 페이지의 전환 없이 바로바로 뭔가 뜨는 것
# 이것이 전형적인 ajax 가 도는 사이트들

In [8]:
resp = get('https://brunch.co.kr/search', params={'q':'한소희'})

In [9]:
resp.status_code, resp.reason, resp.headers
# charset 확인

(200,
 '',
 {'Date': 'Mon, 20 Mar 2023 13:54:18 GMT', 'Content-Type': 'text/html;charset=utf-8', 'Transfer-Encoding': 'chunked', 'Connection': 'keep-alive', 'Content-Language': 'en-US', 'Strict-Transport-Security': 'max-age=15724800; includeSubDomains'})

In [10]:
dom = BeautifulSoup(resp.text, 'html.parser')

In [11]:
dom.select('#resultArticle')
# 원래는 그 밑의 자손 요소들 다 출력되었어야 함. 근데 안됨. 브라우저 돔에서는 보이는데 코드로는 보이지 않음.
# 이거는 결과가 없어서 그렇다. => 즉 ajax 로 돌았다! 라고 예측할 수 있어야 함

[<div class="search_result" id="resultArticle"></div>]

In [12]:
# ajax 를 찾아야 함.
dom.select('form')

[<form action="/search" method="get">
 <input autocomplete="off" id="searchQuery" name="q" placeholder="검색어를 입력해 주세요" title="검색어입력" type="text" value="한소희"/>
 <input name="profileId" type="hidden" value=""/>
 </form>,
 <form accept-charset="utf-8" action="" method="get">
 <input autocomplete="off" class="txt_search" id="txt_search" maxlength="20" name="q" onfocus="this.value = this.value;" placeholder="검색어를 입력해 주세요." title="검색어입력" type="text" value="한소희"/>
 <input id="hdn_search_type" name="type" type="hidden" value="article"/>
 </form>]

In [16]:
dom.select_one('form').attrs

{'action': '/search', 'method': 'get'}

In [17]:
dom.select('form > input')

[<input autocomplete="off" id="searchQuery" name="q" placeholder="검색어를 입력해 주세요" title="검색어입력" type="text" value="한소희"/>,
 <input name="profileId" type="hidden" value=""/>,
 <input autocomplete="off" class="txt_search" id="txt_search" maxlength="20" name="q" onfocus="this.value = this.value;" placeholder="검색어를 입력해 주세요." title="검색어입력" type="text" value="한소희"/>,
 <input id="hdn_search_type" name="type" type="hidden" value="article"/>]

In [18]:
for tag in dom.select('form > input[name]'):
    print(tag.attrs['name'], tag.attrs['value'])

q 한소희
profileId 
q 한소희
type article


In [19]:
resp.url

'https://brunch.co.kr/search?q=%ED%95%9C%EC%86%8C%ED%9D%AC'

In [ ]:
# 항상 맨 처음은 html.
# 그 문서를 따라 필요한 리소스부터 차례대로 접근
# XHR 이 실질적으로 데이터를 담고 ajax 로 보내는 애
# XHR 의 요청 헤더를 보면 주소를 얻을 수 있음(어디로 보내는지)

In [20]:
'/v1/search/article'
from urllib.parse import parse_qs
parse_qs('q=%ED%95%9C%EC%86%8C%ED%9D%AC&page=1&pageSize=20&highlighter=y&escape=y&sortBy=accu')

{'q': ['한소희'],
 'page': ['1'],
 'pageSize': ['20'],
 'highlighter': ['y'],
 'escape': ['y'],
 'sortBy': ['accu']}

In [21]:
from requests.compat import urljoin

url = urljoin(resp.url, '/v1/search/article')
params = {'q': '한소희',
 'page': '1',
 'pageSize': '20',
 'highlighter': 'y',
 'escape': 'y',
 'sortBy': 'accu'}

In [22]:
resp = get(url, params=params)

In [23]:
resp.text
# 결과를 주지 않음

'<!DOCTYPE html>\n<html lang="ko">\n<head>\n\t<meta charset="UTF-8">\n\t<meta name="viewport" content="user-scalable=no, initial-scale=1.0, maximum-scale=1.0, minimum-scale=1.0, width=device-width">\n\t<title>brunch</title>\n\t\n\n\n\n<title>brunch</title>\n<meta charset="utf-8">\n<meta http-equiv="Content-Type" content="text/html; charset=utf-8">\n<meta http-equiv="X-UA-Compatible" content="IE=edge">\n<meta name="title" content="brunch">\n<meta name="referrer" content="always">\n<meta name="secureToken" value="">\n<meta name="robots" content="max-image-preview:large">\n\n\n<link rel="SHORTCUT ICON" href="//t1.daumcdn.net/brunch/static/icon/favicon/favicon64_150520.ico"/>\n<link rel="apple-touch-icon-precomposed" sizes="180x180" href="//t1.daumcdn.net/brunch/static/icon/ios/icon152.png">\n<link rel="apple-touch-icon-precomposed" sizes="152x152" href="//t1.daumcdn.net/brunch/static/icon/ios/icon152.png">\n<link rel="apple-touch-icon-precomposed" sizes="120x120" href="//t1.daumcdn.net/br

In [24]:
# 헤더를 넣어보자
url = 'https://api.brunch.co.kr/v1/search/article'
resp = get(url, params=params, headers=headers)

In [25]:
resp.headers
# 실제 브라우저의 응답의 콘텐트 타입(개발자도구에서) 을 보고 잘 가져왔는지 확인 가능

{'Date': 'Mon, 20 Mar 2023 14:03:36 GMT', 'Content-Type': 'application/json;charset=UTF-8', 'Transfer-Encoding': 'chunked', 'Connection': 'keep-alive', 'Cache-Control': 'no-cache, no-store, must-revalidate, no-store', 'Pragma': 'no-cache', 'Expires': 'Thu, 01 Jan 1970 00:00:00 GMT', 'Access-Control-Allow-Methods': 'GET,POST,HEAD,OPTIONS,PUT,DELETE', 'Access-Control-Allow-Headers': 'Content-Type,X-Requested-With,accept,Origin,Access-Control-Request-Method,Access-Control-Request-Headers,brunch-client-os,X-CSRF-TOKEN,X-JSESSIONID,referer,brunch-agent', 'Access-Control-Allow-Credentials': 'true', 'Set-Cookie': 'bid=""; Domain=.brunch.co.kr; Expires=Thu, 01-Jan-1970 00:00:10 GMT; Path=/; Secure', 'Content-Language': 'en-US', 'Strict-Transport-Security': 'max-age=15724800; includeSubDomains'}

In [26]:
resp.text

'{"desc":"OK","code":200,"title":"","extra":"","data":{"recommendProfiles":[{"userId":"18al","userName":"songoflucia","userImage":"http://t1.daumcdn.net/brunch/service/guest/image/wVZS5AUETvS0bmSE2o7osKTOmdo.png","profileId":"songoflucia","profileImage":null,"description":"모든 게 다 그렇지는 않지만 영화를 보다가 종종 ‘나’를 돌아봅니다. 그리고 이젠 엄마가 되었습니다.","createTime":1453722294000,"status":"verify","articleCount":192,"socialUser":false,"writerCount":16,"followerCount":1738,"magazineCount":0,"company":"","descriptionDetail":"영화로는 꿈을 꾸고, 마케팅으로는 현실을 살고 있는 반도의 흔한 온라인 마케터. 영화와 관련된 글을 꾸준히 쓰고 싶어서 계속 고민하다가 ‘brunch’라는 공간을 발견했고, 2017년부터 브런치 무비패스를 통해 지속적으로 글을 쓰고 있는 중이다.\\n\\n- 에코 라이프 매거진 그린마인드 5호~8호, 11호~14호 ‘영화가 돌아왔다’ 기고\\n- 영화비평매거진 CAST 3호, 4호, 5호, 6호, 7호 기고","profileNotiExpireTime":1667653871000,"userSns":{"userId":"18al","website":"","facebook":"","twitter":"","instagram":"","agreeSns":false,"createTime":1533188978000,"updateTime":1667049071000},"bookStoreBookList":[],"profileCategoryList":null,"mainProduct":null,"ac

In [27]:
resp.json()

{'desc': 'OK',
 'code': 200,
 'title': '',
 'extra': '',
 'data': {'recommendProfiles': [{'userId': '18al',
    'userName': 'songoflucia',
    'userImage': 'http://t1.daumcdn.net/brunch/service/guest/image/wVZS5AUETvS0bmSE2o7osKTOmdo.png',
    'profileId': 'songoflucia',
    'profileImage': None,
    'description': '모든 게 다 그렇지는 않지만 영화를 보다가 종종 ‘나’를 돌아봅니다. 그리고 이젠 엄마가 되었습니다.',
    'createTime': 1453722294000,
    'status': 'verify',
    'articleCount': 192,
    'socialUser': False,
    'writerCount': 16,
    'followerCount': 1738,
    'magazineCount': 0,
    'company': '',
    'descriptionDetail': '영화로는 꿈을 꾸고, 마케팅으로는 현실을 살고 있는 반도의 흔한 온라인 마케터. 영화와 관련된 글을 꾸준히 쓰고 싶어서 계속 고민하다가 ‘brunch’라는 공간을 발견했고, 2017년부터 브런치 무비패스를 통해 지속적으로 글을 쓰고 있는 중이다.\n\n- 에코 라이프 매거진 그린마인드 5호~8호, 11호~14호 ‘영화가 돌아왔다’ 기고\n- 영화비평매거진 CAST 3호, 4호, 5호, 6호, 7호 기고',
    'profileNotiExpireTime': 1667653871000,
    'userSns': {'userId': '18al',
     'website': '',
     'facebook': '',
     'twitter': '',
     'instagram': '',
     'a

In [28]:
resp.json()['data']

{'recommendProfiles': [{'userId': '18al',
   'userName': 'songoflucia',
   'userImage': 'http://t1.daumcdn.net/brunch/service/guest/image/wVZS5AUETvS0bmSE2o7osKTOmdo.png',
   'profileId': 'songoflucia',
   'profileImage': None,
   'description': '모든 게 다 그렇지는 않지만 영화를 보다가 종종 ‘나’를 돌아봅니다. 그리고 이젠 엄마가 되었습니다.',
   'createTime': 1453722294000,
   'status': 'verify',
   'articleCount': 192,
   'socialUser': False,
   'writerCount': 16,
   'followerCount': 1738,
   'magazineCount': 0,
   'company': '',
   'descriptionDetail': '영화로는 꿈을 꾸고, 마케팅으로는 현실을 살고 있는 반도의 흔한 온라인 마케터. 영화와 관련된 글을 꾸준히 쓰고 싶어서 계속 고민하다가 ‘brunch’라는 공간을 발견했고, 2017년부터 브런치 무비패스를 통해 지속적으로 글을 쓰고 있는 중이다.\n\n- 에코 라이프 매거진 그린마인드 5호~8호, 11호~14호 ‘영화가 돌아왔다’ 기고\n- 영화비평매거진 CAST 3호, 4호, 5호, 6호, 7호 기고',
   'profileNotiExpireTime': 1667653871000,
   'userSns': {'userId': '18al',
    'website': '',
    'facebook': '',
    'twitter': '',
    'instagram': '',
    'agreeSns': False,
    'createTime': 1533188978000,
    'updateTime': 1667049071000},
   

In [29]:
from html import unescape

for li in resp.json()['data']['list']:
    print(unescape(li['title']))

날것의 <b>한소희</b>! - 드라마 <마이네임> 리뷰
네오 팜므파탈의 아우라를 지닌 <b>한소희</b> - <b>한소희</b> said “My Name is Neo Femme Fatale!”
나의 중고차 판매기-1 - 왜 헤이딜러는 김혜수, <b>한소희</b>가 광고할까?
그래피티, 함께라는 이름으로 - HBAF — <b>한소희</b> X 바프 Graffiti by LEODAV
[명대사]드라마, 알고 있지만. - <송블리의 한 줄 명대사중계> | 송강, <b>한소희</b>.
“저기 <b>한소희</b> 씨, 그 차 나이가 더 많아요” - 로터스 에스프리 1988년형
<b>한소희</b>가 곱슬머리면 좋겠다.
꼭 그래야만 했습니까? - feat. <b>한소희</b> 배우의 엄마를 보고
[하루의기적] <b>한소희</b>로 시작해서 나로 끝나는 글
다시 태어난다면 - 전지현도 아니고 <b>한소희</b>도 아니다
<b>한소희</b>는 연기력을 쌓은 다음 편안한 멜로를 찍는게
마이 네임 <b>한소희</b> 단발
[넷플릭스] 마이 네임 (2021) - 이름과 인생을 갖다바친 최후의 복수 (<b>한소희</b>/안보현/넷플릭스 드라마)
배우 <b>한소희</b>, 옷가게, 고깃집, 호프집까지 알바나선이유 - 배우 <b>한소희</b>, 옷가게, 고깃집, 호프집까지 않해 본 알바가 없어요.
빙그레 따옴 + <b>한소희</b> - 2020년 5월 29일 따옴 광고 온에어 (JTBC)
<송강> 넷플릭스의 아들, 나빌레라의 주인공 - 미래는 <b>한소희</b>의 남자
'알고있지만' <b>한소희</b>의 러블리한 매력을 닮은 여름 부케 - 러블리 상큼 향기로운 여름 부케 추천!
블랙핑크 지수랑 같은 이름이네요? 나 완전 팬인데! - 런던에서 맥주 따르기 8번째
3-1. 프랑스 1일차
요즘 본 OTT 드라마들 - - 요즘 그렇게 드라마를 본다.


In [ ]:
# 실습. 네이버 검색창 자동완성 기능 가져오기 - 글자는 '한소희' 치고
# 검색창에 한소희 치니까 개발자도구 network에서 자바스크립트 문서가 계속 반복적으로 올라옴

In [ ]:
https://ac.search.naver.com/nx/ac?q=%ED%95%9C%EC%86%8C%ED%9D%AC&con=0&frm=nv&ans=2&r_format=json&r_enc=UTF-8&r_unicode=0&t_koreng=1&run=2&rev=4&q_enc=UTF-8&st=100&_callback=_jsonp_11

In [30]:
parse_qs('q=%ED%95%9C%EC%86%8C%ED%9D%AC&con=0&frm=nv&ans=2&r_format=json&r_enc=UTF-8&r_unicode=0&t_koreng=1&run=2&rev=4&q_enc=UTF-8&st=100&_callback=_jsonp_11')

{'q': ['한소희'],
 'con': ['0'],
 'frm': ['nv'],
 'ans': ['2'],
 'r_format': ['json'],
 'r_enc': ['UTF-8'],
 'r_unicode': ['0'],
 't_koreng': ['1'],
 'run': ['2'],
 'rev': ['4'],
 'q_enc': ['UTF-8'],
 'st': ['100'],
 '_callback': ['_jsonp_11']}

In [31]:
url = 'https://ac.search.naver.com/nx/ac'
params = {'q': '한소희',
 'con': '0',
 'frm': 'nv',
 'ans': '2',
 'r_format': 'json',
 'r_enc': 'UTF-8',
 'r_unicode': '0',
 't_koreng': '1',
 'run': '2',
 'rev': '4',
 'q_enc': 'UTF-8',
 'st': '100'}
# _callback 은 얘를 해석할 자바스크립트 함수를 나타냄. 이거 붙히고 header 찍어보면 content-type 에 자바스크립트 나올 것
params['q'] = '한'
resp = get(url, params=params, headers=headers)
resp.json()['items']

[[['한소희', '0'],
  ['한블리', '0'],
  ['한 페이지가 될 수 있게', '0'],
  ['한동훈', '0'],
  ['한국사능력검정시험', '1'],
  ['하나은행', '0'],
  ['한자사전', '0'],
  ['하나투어', '0'],
  ['한섬', '0'],
  ['한유진', '0']]]

In [32]:
resp = get(url, params=params, headers=headers)

In [34]:
resp.headers
# 'Content-Type': 'application/json => 결과 잘 가져왔음을 확인
# 개발자도구 네트워크에서 해당 자바스크립트 헤더의 응답 부분 content-type 확인

{'Date': 'Mon, 20 Mar 2023 14:09:27 GMT', 'Content-Type': 'application/json; charset=UTF-8', 'Content-Length': '280', 'Connection': 'keep-alive', 'Vary': 'Accept-Encoding', 'Content-Encoding': 'gzip', 'Server': 'nxfps'}

In [35]:
resp.text

'{\n"query" : ["한"],\n"answer" : [],\n"intend" : [{"query":"하니","intend":"캐릭터","transQuery":"달려라하니"},{"query":"하니","intend":"가수, EXID","transQuery":"exid하니"}],\n"items" : [\n[["한소희", "0"],["한블리", "0"],["한 페이지가 될 수 있게", "0"],["한동훈", "0"],["한국사능력검정시험", "1"],["하나은행", "0"],["한자사전", "0"],["하나투어", "0"],["한섬", "0"],["한유진", "0"]]\n]\n}'

In [36]:
resp.json()['items']

[[['한소희', '0'],
  ['한블리', '0'],
  ['한 페이지가 될 수 있게', '0'],
  ['한동훈', '0'],
  ['한국사능력검정시험', '1'],
  ['하나은행', '0'],
  ['한자사전', '0'],
  ['하나투어', '0'],
  ['한섬', '0'],
  ['한유진', '0']]]

In [37]:
url = 'https://ac.search.naver.com/nx/ac'
params = {'q': '한소희',
 'con': '0',
 'frm': 'nv',
 'ans': '2',
 'r_format': 'json',
 'r_enc': 'UTF-8',
 'r_unicode': '0',
 't_koreng': '1',
 'run': '2',
 'rev': '4',
 'q_enc': 'UTF-8',
 'st': '100'}
# params['q'] 에다가 우리가 치면 자동완성 기능 구현 가능
params['q'] = '한소'
resp = get(url, params=params, headers=headers)
resp.json()['items']

[[['한소희', '0'],
  ['한솥도시락', '0'],
  ['한솥', '0'],
  ['한소희 가방', '0'],
  ['한소희 블로그', '0'],
  ['한솔아카데미', '0'],
  ['한솔', '0'],
  ['한송이', '0'],
  ['한솔케미칼', '0'],
  ['한솔제지', '0']]]

In [38]:
dom = BeautifulSoup(get('https://www.naver.com').text, 'html.parser')
dom.select('form')

[<form action="https://search.naver.com/search.naver" id="sform" method="get" name="sform" role="search">
 <fieldset>
 <legend class="blind">검색</legend>
 <input name="where" type="hidden" value="nexearch"/>
 <input id="sm" name="sm" type="hidden" value="top_hty"/>
 <input id="fbm" name="fbm" type="hidden" value="0"/>
 <input disabled="disabled" id="acr" name="acr" type="hidden" value=""/>
 <input disabled="disabled" id="acq" name="acq" type="hidden" value=""/>
 <input disabled="disabled" id="qdt" name="qdt" type="hidden" value=""/>
 <input id="ie" name="ie" type="hidden" value="utf8"/>
 <input disabled="disabled" id="acir" name="acir" type="hidden" value=""/>
 <input disabled="disabled" id="os" name="os" type="hidden" value=""/>
 <input disabled="disabled" id="bid" name="bid" type="hidden" value=""/>
 <input disabled="disabled" id="pkid" name="pkid" type="hidden" value=""/>
 <input disabled="disabled" id="eid" name="eid" type="hidden" value=""/>
 <input disabled="disabled" id="mra" nam

#### 웹툰 가져오기

In [ ]:
# 이전에 실습에서 HTML DOM 으로 얻어올 수 있는 정보가 제한적임을 알았음
# 그렇다면 뒷단에서 ajax 로 json 을 받아왔겠구나! 생각

In [ ]:
https://comic.naver.com/webtoon/detail?titleId=648419&no=377
# 뒷단에서 ajax 보내는 애
titleId=648419&no=377 # 파라미터
titleId=648419 <= 웹툰 (weekday)
no=377 <= 회차(list?웹툰키값)
# 어디서 얻었을까?

In [41]:
# 메타정보 받아오기 XHR 에서
# 웹툰 고유 ID, 웹툰별 회차 ID
url1 = 'https://comic.naver.com/api/webtoon/titlelist/weekday?order=user'
resp = get(url1)

In [42]:
resp.headers

{'date': 'Mon, 20 Mar 2023 14:40:06 GMT', 'content-type': 'application/json', 'transfer-encoding': 'chunked', 'set-cookie': 'XSRF-TOKEN=37189730-8e01-42aa-a96f-286a73a6ff63; Path=/', 'x-content-type-options': 'nosniff', 'x-xss-protection': '1; mode=block', 'cache-control': 'no-cache, no-store, max-age=0, must-revalidate', 'pragma': 'no-cache', 'expires': '0', 'x-frame-options': 'SAMEORIGIN', 'vary': 'accept-encoding', 'content-encoding': 'gzip', 'referrer-policy': 'unsafe-url', 'server': 'nfront'}

In [43]:
resp.json()

{'titleListMap': {'WEDNESDAY': [{'titleId': 747269,
    'titleName': '전지적 독자 시점',
    'author': 'UMI / 슬리피-C / 싱숑',
    'thumbnailUrl': 'https://image-comic.pstatic.net/webtoon/747269/thumbnail/thumbnail_IMAG21_aabd9952-ff45-47a2-a543-33f19a5c6708.jpg',
    'up': False,
    'rest': False,
    'bm': False,
    'adult': False,
    'starScore': 9.93641,
    'viewCount': 0,
    'openToday': False,
    'potenUp': False,
    'bestChallengeLevelUp': False,
    'finish': False,
    'new': False},
   {'titleId': 670143,
    'titleName': '헬퍼 2 : 킬베로스',
    'author': '삭',
    'thumbnailUrl': 'https://image-comic.pstatic.net/webtoon/670143/thumbnail/thumbnail_IMAG21_12d5d293-d54d-4022-8e00-c97c5779c701.jpg',
    'up': False,
    'rest': False,
    'bm': False,
    'adult': True,
    'starScore': 8.74156,
    'viewCount': 59603070,
    'openToday': False,
    'potenUp': False,
    'bestChallengeLevelUp': False,
    'finish': False,
    'new': False},
   {'titleId': 804862,
    'titleName': '백XX',
 

In [44]:
resp.json()['titleListMap'].items()

dict_items([('WEDNESDAY', [{'titleId': 747269, 'titleName': '전지적 독자 시점', 'author': 'UMI / 슬리피-C / 싱숑', 'thumbnailUrl': 'https://image-comic.pstatic.net/webtoon/747269/thumbnail/thumbnail_IMAG21_aabd9952-ff45-47a2-a543-33f19a5c6708.jpg', 'up': False, 'rest': False, 'bm': False, 'adult': False, 'starScore': 9.93641, 'viewCount': 0, 'openToday': False, 'potenUp': False, 'bestChallengeLevelUp': False, 'finish': False, 'new': False}, {'titleId': 670143, 'titleName': '헬퍼 2 : 킬베로스', 'author': '삭', 'thumbnailUrl': 'https://image-comic.pstatic.net/webtoon/670143/thumbnail/thumbnail_IMAG21_12d5d293-d54d-4022-8e00-c97c5779c701.jpg', 'up': False, 'rest': False, 'bm': False, 'adult': True, 'starScore': 8.74156, 'viewCount': 59603070, 'openToday': False, 'potenUp': False, 'bestChallengeLevelUp': False, 'finish': False, 'new': False}, {'titleId': 804862, 'titleName': '백XX', 'author': '박태준 만화회사, 병장 / 펀치킥', 'thumbnailUrl': 'https://image-comic.pstatic.net/webtoon/804862/thumbnail/thumbnail_IMAG21_e2814

In [45]:
webtoon = list()
for k,v in resp.json()['titleListMap'].items():
#    print(k, len(v))
    for li in v:
#        print(li['titleId'], li['titleName'])
         webtoon.append((li['titleId'], li['titleName']))

In [46]:
len(webtoon)

581

In [47]:
v

[{'titleId': 736277,
  'titleName': '싸움독학',
  'author': '박태준 만화회사 / 김정현 스튜디오',
  'thumbnailUrl': 'https://image-comic.pstatic.net/webtoon/736277/thumbnail/thumbnail_IMAG21_bbbe3f76-021e-4dbc-830a-4358c1abec0c.jpg',
  'up': False,
  'rest': False,
  'bm': False,
  'adult': False,
  'starScore': 9.71996,
  'viewCount': 0,
  'openToday': False,
  'potenUp': False,
  'bestChallengeLevelUp': False,
  'finish': False,
  'new': False},
 {'titleId': 710751,
  'titleName': '약한영웅',
  'author': '서패스 / 김진석',
  'thumbnailUrl': 'https://image-comic.pstatic.net/webtoon/710751/thumbnail/thumbnail_IMAG21_53aefc06-6bdf-40fb-93ac-fab7242146c6.jpg',
  'up': False,
  'rest': False,
  'bm': False,
  'adult': False,
  'starScore': 9.7951,
  'viewCount': 10409921,
  'openToday': False,
  'potenUp': False,
  'bestChallengeLevelUp': False,
  'finish': False,
  'new': False},
 {'titleId': 774044,
  'titleName': '투신전생기',
  'author': '청담',
  'thumbnailUrl': 'https://image-comic.pstatic.net/webtoon/774044/thumbnail

In [48]:
# 메타정보 받아오기 xhr 에서
# 웹툰 고유 ID, 웹툰별 회차 ID
url1 = 'https://comic.naver.com/api/webtoon/titlelist/weekday?order=user'  # 요일별 웹툰 목록 페이지
url2 = 'https://comic.naver.com/api/article/list/info'  # 각 웹툰의 정보
url3 = 'https://comic.naver.com/api/article/list' # 각 웹툰의 회차
url4 = 'https://comic.naver.com/webtoon/detail' # 각 웹툰의 특정 회차 들어갔을 때의 주소

params2 = {'titleId':None}
params3 = {'titleId':None, 'page':1}
params4 = {'titleId':None, 'no':None}

In [49]:
# 웹툰 목록을 다 돌면서 회차 정보 가져오고, 각 웹툰의 회차
URLs = [url1]
visited = list()

#while URLs:
for _ in range(10):
    seed = URLs.pop(-1)  # stack(뒤에부터 가져오기)
    visited.append(seed)
    
    resp = get(seed, headers=headers)
    
    if resp.status_code != 200:
        continue
        
    if re.search('image', resp.headers['content-type']):
        filename = resp.url.split('/')[-1]
        filename = re.sub('[?#!]', '', filename)
        ext = re.search('image/(\w+);?',resp.headers['content-type']).group(1)
        with open('./webtoon/'+filename+'.'+ext, 'wb') as fp:
            fp.write(resp.content)
        
    # 뒷단에서 XHR 통신을 하면 json을 받음
    if re.search('json', resp.headers['content-type']): 
        data = resp.json()  # 이 때부터 키밸류 접근 가능(python dict)
        # 영역 제한 -1  (웹툰 목록 홈)
        if 'titleListMap' in data.keys():
            for k, v in data['titleListMap'].items():
                for li in v:  # MONDAY:[li:... li:...]
                    nurl = 'https://comic.naver.com/api/article/list?titleId={}&page=1'.format(li['titleId'])
                    if nurl not in URLs and nurl not in visited:
                        URLs.append(nurl)
                        #URLs.append(f'https://comic.naver.com/api/article/list?titleId={}&page=1', format(li['titleId']))
                        # 방문할 곳들을 쭉 추가해줌
                
        # 영역 제한 -2  (특정 웹툰의 회차 목록)
        if 'articleList' in data.keys():
            for li in data['articleList']:
                # 실제 웹툰이 보이는 주소
                nurl = 'https://comic.naver.com/webtoon/detail?titleId={}&no={}'.format(data['titleId'], li['no'])
                if nurl not in URLs and nurl not in visited:
                    URLs.append(nurl)
                    #URLs.append(f'https://comic.naver.com/webtoon/detail?titleId={data['titleId']}&no={li['no']}')
                    # 앞으로 방문할 목록이 만들어짐
                        
    if re.search('html', resp.headers['content-type']):
        dom = BeautifulSoup(resp.text, 'html.parser')
        
        # 영역 제한 -3  (특정 웹툰의 특정 회차의 이미지 목록)
        for img in dom.select('img[id^=content_image_]'):
            nurl = urljoin(seed, img.attrs['src'])
            if nurl not in URLs and nurl not in visited:
                URLs.append(nurl)

In [50]:
URLs

['https://comic.naver.com/api/article/list?titleId=783053&page=1',
 'https://comic.naver.com/api/article/list?titleId=790713&page=1',
 'https://comic.naver.com/api/article/list?titleId=784248&page=1',
 'https://comic.naver.com/api/article/list?titleId=703846&page=1',
 'https://comic.naver.com/api/article/list?titleId=796152&page=1',
 'https://comic.naver.com/api/article/list?titleId=730656&page=1',
 'https://comic.naver.com/api/article/list?titleId=743139&page=1',
 'https://comic.naver.com/api/article/list?titleId=789979&page=1',
 'https://comic.naver.com/api/article/list?titleId=774358&page=1',
 'https://comic.naver.com/api/article/list?titleId=804783&page=1',
 'https://comic.naver.com/api/article/list?titleId=738487&page=1',
 'https://comic.naver.com/api/article/list?titleId=801589&page=1',
 'https://comic.naver.com/api/article/list?titleId=711422&page=1',
 'https://comic.naver.com/api/article/list?titleId=719508&page=1',
 'https://comic.naver.com/api/article/list?titleId=802389&page

In [51]:
len(URLs), len(visited)

(631, 10)

In [52]:
!ls webtoon/

20220925103402_210e710147193378100ed43d2dcc1a0a_IMAG01_45.jpg.jpeg
20220925103402_210e710147193378100ed43d2dcc1a0a_IMAG01_46.jpg.jpeg
20220925103402_210e710147193378100ed43d2dcc1a0a_IMAG01_47.jpg.jpeg
20220925103402_210e710147193378100ed43d2dcc1a0a_IMAG01_48.jpg.jpeg
20220925103402_210e710147193378100ed43d2dcc1a0a_IMAG01_49.jpg.jpeg
20220925103402_210e710147193378100ed43d2dcc1a0a_IMAG01_50.jpg.jpeg
20220925103402_210e710147193378100ed43d2dcc1a0a_IMAG01_51.jpg.jpeg


In [ ]:
# 크롤러의 주 기능은 링크 단위로 움직이는 것.
# 특정 이미지들만 가져오겠다고 하면 스크래핑 기능만 있으면 됨
# 꼭 위처럼 안해도 된다는 것.

#### 쿠키와 세션

In [ ]:
# Cookie, Session  => 임시로 데이터를 저장하는 것.
# ; 주요목적이 임시로 데이터 저장. 이 때 Expired Data 라고 해서 수명이 항상 존재함
# 쿠키는 예를 들어 아이디 저장 등
# 즉 쿠키를 이용할 줄 알면 로그인을 할 수 있음.
# 쿠키를 다룰 줄 알면 상대방의 쿠키정보를 이용해서 내가 대신 로그인 한 것처럼 이용할 수 있음

Client               Server
 req ---------------->
 resp <---------------
                           이게 HTTP(TCP/IP)

문제는 서버에서 클라이언트를 기억함 -> 요청/응답이면 끝나는데 어떻게 나를 기억하고 있을까?
그게 쿠키와 세션 때문에 가능!
특히 쿠키는 클라이언트에 저장되는 정보를 통칭해서 쿠키라고 함
서버에 저장되는 것을 세션이라고 함

[Login] ------------> (ID/PW)
  OK    <------------ (마이페이지 같은 거 보여주면서)
                      여기서 서버가 내가 누군지 알고 있음(서버에서 관리함)
                      비용적 측면 때문에 쿠키에 정보를 심음
 mail  ------------>
 내 메일  <------------

In [53]:
url = 'https://pythonscraping.com/pages/cookies/login.html'
resp = get(url)
print(resp.text)

<h2>Log In Here!</h2>
<form method="post" action="welcome.php">
Username (use anything!): <input type="text" name="username"><br>
Password (try "password"): <input type="password" name="password"><br>
<input type="submit" value="Login">
</form>


In [54]:
dom = BeautifulSoup(resp.text, 'html.parser')

In [55]:
dom.select_one('form')

<form action="welcome.php" method="post">
Username (use anything!): <input name="username" type="text"/><br/>
Password (try "password"): <input name="password" type="password"/><br/>
<input type="submit" value="Login"/>
</form>

In [56]:
dom.select_one('form').attrs

{'method': 'post', 'action': 'welcome.php'}

In [57]:
for li in dom.select('form input[name]'):
    print(li.attrs)

{'type': 'text', 'name': 'username'}
{'type': 'password', 'name': 'password'}


In [58]:
urljoin(url, dom.select_one('form').attrs['action'])

'https://pythonscraping.com/pages/cookies/welcome.php'

In [59]:
from requests import post

In [60]:
resp = post(urljoin(url, dom.select_one('form').attrs['action']),
           data={'username':'asdfasdfas', 'password':'password'})

In [61]:
resp.status_code, resp.request.body, resp.headers

(200,
 'username=asdfasdfas&password=password',
 {'Server': 'nginx', 'Date': 'Mon, 20 Mar 2023 14:47:49 GMT', 'Content-Type': 'text/html; charset=UTF-8', 'Content-Length': '147', 'Connection': 'keep-alive', 'X-Powered-By': 'PHP/7.4.33, PleskLin', 'Set-Cookie': 'loggedin=1, username=asdfasdfas', 'Vary': 'Accept-Encoding', 'Content-Encoding': 'gzip'})

In [62]:
resp.headers['set-cookie']

'loggedin=1, username=asdfasdfas'

In [63]:
print(resp.text)


<h2>Welcome to the Website!</h2>
Whoops! You logged in wrong. Try again with any username, and the password "password"<br><a href="login.html">Log in here</a>


In [64]:
from requests import request, post
from requests.cookies import create_cookie
from requests.compat import urlencode

In [ ]:
request()
# :param cookies: (optional) Dict or CookieJar object to send with the :class:`Request`.

In [65]:
resp = post(urljoin(url, dom.select_one('form').attrs['action']),
           data={'username':'asdfasdfas', 'password':'password'},
           cookies={'loggedin':1, 'username':'asdasdsada'})

TypeError: expected string or bytes-like object

In [66]:
cookie = create_cookie('loggedin', 1)

In [91]:
cookie

Cookie(version=0, name='loggedin', value=1, port=None, port_specified=False, domain='', domain_specified=False, domain_initial_dot=False, path='/', path_specified=True, secure=False, expires=None, discard=True, comment=None, comment_url=None, rest={'HttpOnly': None}, rfc2109=False)

In [68]:
resp = post(urljoin(url, dom.select_one('form').attrs['action']),
           data={'username':'asdfasdfas', 'password':'password'},
           cookies=cookie)

TypeError: 'Cookie' object is not iterable

In [69]:
resp = post(urljoin(url, dom.select_one('form').attrs['action']),
           data={'username':'asdfasdfas', 'password':'password'},
           cookies=[cookie])

TypeError: list indices must be integers or slices, not Cookie

In [70]:
resp.status_code, resp.request.body

(200, 'username=asdfasdfas&password=password')

In [71]:
resp.headers['set-cookie']

'loggedin=1, username=asdfasdfas'

In [72]:
print(resp.text)


<h2>Welcome to the Website!</h2>
Whoops! You logged in wrong. Try again with any username, and the password "password"<br><a href="login.html">Log in here</a>


위에 잘 안되니까 Session 으로 접근

In [87]:
from requests.sessions import Session

In [88]:
session = Session()
# request 객체랑 똑같은데(요청/응답 받음) 다만 쿠키를 들고 있음

In [92]:
session.cookies.set('loggedin', '1')
session.cookies.set('username', 'ssadasdfa')

Cookie(version=0, name='username', value='ssadasdfa', port=None, port_specified=False, domain='', domain_specified=False, domain_initial_dot=False, path='/', path_specified=True, secure=False, expires=None, discard=True, comment=None, comment_url=None, rest={'HttpOnly': None}, rfc2109=False)

In [93]:
resp = session.post(urljoin(url, dom.select_one('form').attrs['action']))

In [95]:
from requests.cookies import cookiejar_from_dict

In [96]:
cookie = cookiejar_from_dict({'loggedin': '1'})

In [100]:
resp = post(urljoin(url, dom.select_one('form').attrs['action']),
           data={'username':'asdfasdfas', 'password':'password'},
           cookies=cookie)

In [98]:
resp = session.post(urljoin(url, dom.select_one('form').attrs['action']),
                    data={'username':'asdfasdfas', 'password':'password'})

In [101]:
get(urljoin(url, dom.select_one('form').attrs['action'])).text

'\n<h2>Welcome to the Website!</h2>\nWhoops! You logged in wrong. Try again with any username, and the password "password"<br><a href="login.html">Log in here</a>'

In [102]:
session.get(urljoin(url, dom.select_one('form').attrs['action'])).text

'\n<h2>Welcome to the Website!</h2>\nYou have logged in successfully! <br><a href="profile.php">Check out your profile!</a>'

In [103]:
get(urljoin(url, dom.select_one('form').attrs['action']),
   cookies=session.cookies).text

'\n<h2>Welcome to the Website!</h2>\nYou have logged in successfully! <br><a href="profile.php">Check out your profile!</a>'

In [ ]:
session 객체를 이용해서 get, post, .....
기존 것이랑 그 결과값이 다름. 차이점은 헤더에 있는 set-cookie 라는 부분
이것을 가지고 cookie jar 객체를 생성하고 다음번에 이용할 수 있게 해줌.
session.get(cookies=쿠키)  => 로그인 OK
get() => 로그인 x, get(cookies=session.cookie) => 로그인 OK

In [104]:
type(session.cookies)

requests.cookies.RequestsCookieJar

In [105]:
session.cookies.set('test', '2314142')

Cookie(version=0, name='test', value='2314142', port=None, port_specified=False, domain='', domain_specified=False, domain_initial_dot=False, path='/', path_specified=True, secure=False, expires=None, discard=True, comment=None, comment_url=None, rest={'HttpOnly': None}, rfc2109=False)

In [106]:
session.cookies.get_dict()

{'loggedin': '1', 'test': '2314142', 'username': 'asdfasdfas'}

In [ ]:
# 세션이 쿠키를 알아서 관리를 해주기 때문에 우리가 신경 안 써도 됨.

In [107]:
# 네이버 쿠키값 복사
# Application -> Cookies
c = '''
NID_JKL	e6zeko9r9guajE5nYu5Uc5VeAFEy/LKT+MO7EyL2TEs=	.naver.com	/	2024-04-23T03:28:30.763Z	51		✓			Medium	
NID_SES	AAABqXaeyYewqNhIIphsO/IQtD9NpY2O/Ng3KUXeOThhELBgRlxGy9gbwK89AVl5Tn6hDTpTN+kOKBc4EN1MYW40oQj2BMOL61hDRsItLVBjFjrW2WExejgGF4Ek2/KFvMtQuvO9N5SrFlLAtc5QM8Wc7cd5D5PVC+Sk26OcMTibz9IXuYcgFKxMzJ+drNjKPOv4qdkuh61xBFz8MBsHMv/N4wfc/nXNFkvRo8EViaSpjoef1cSq9dkxZdFtvfiay0oq8dhUA934+S+rNvjic1RmW6CQStKgjgANAyjYX23O9JfzZDGm7TEOAGI8xRQkUNtuCd5zVsCiPVFhPMpCTp6w2OUj+8Hy0u5L1d0g0+jN9YP/QPkI3xUcKNvJIkpcAd3ubxRJH2s1DryTA6KBecQlUSF2GTRAzjtnDTKdOIWI2ZdTt5vK3wTQ7noVL/xsbI5AJgO8ScQK9F/85Q0PBDnp1hjQOQnMtBAqtWjeNIYKSZA1qqTP8nJGczt8H4j+Z+Px3xVOEefV6fZqe15auvIMR9rDg3PSI/dcYLmbHDzo6rXvk0yGT7DgKN4rVdL0Apgp1Q==	.naver.com	/	2023-04-19T03:28:31.767Z	591					Medium	
NID_AUT	72TP2XCZib5da7eG8p6g9PXUrwtbhxknB5uPTfAtH4/QXI20z/sTVAz4LCtxb73N	.naver.com	/	2024-04-23T03:28:30.763Z	71	✓				Medium	
page_uid	iteB2wprvmZssnKqygRssssssgd-126275	.naver.com	/	Session	42					Medium	
nid_inf	1075385671	.naver.com	/	2023-04-19T03:28:30.763Z	17					Medium	
NNB	YPL7KNNXW3UWG	.naver.com	/	2024-04-23T03:28:31.351Z	16		✓	None		Medium	
nx_ssl	2	.naver.com	/	2023-03-25T15:37:17.878Z	7					Medium	
PM_CK_loc	effb43a101d8bd43219895870fe7f4b34ce7dc3325945658bf9f7d1dcfacae4e	www.naver.com	/	2023-03-21T00:19:57.081Z	73	✓				Medium	
SB_MODE	plusdeal	www.naver.com	/	Session	15					Medium	
'''

# [l.split()[:2] for l in splitlines() if len(l) > 1]
# 우리에게 필요한 것은 키와 밸류

In [108]:
session.cookies.clear()

In [109]:
[l.split()[:2] for l in c.splitlines() if len(l) > 1]

[['NID_JKL', 'e6zeko9r9guajE5nYu5Uc5VeAFEy/LKT+MO7EyL2TEs='],
 ['NID_SES',
  'AAABqXaeyYewqNhIIphsO/IQtD9NpY2O/Ng3KUXeOThhELBgRlxGy9gbwK89AVl5Tn6hDTpTN+kOKBc4EN1MYW40oQj2BMOL61hDRsItLVBjFjrW2WExejgGF4Ek2/KFvMtQuvO9N5SrFlLAtc5QM8Wc7cd5D5PVC+Sk26OcMTibz9IXuYcgFKxMzJ+drNjKPOv4qdkuh61xBFz8MBsHMv/N4wfc/nXNFkvRo8EViaSpjoef1cSq9dkxZdFtvfiay0oq8dhUA934+S+rNvjic1RmW6CQStKgjgANAyjYX23O9JfzZDGm7TEOAGI8xRQkUNtuCd5zVsCiPVFhPMpCTp6w2OUj+8Hy0u5L1d0g0+jN9YP/QPkI3xUcKNvJIkpcAd3ubxRJH2s1DryTA6KBecQlUSF2GTRAzjtnDTKdOIWI2ZdTt5vK3wTQ7noVL/xsbI5AJgO8ScQK9F/85Q0PBDnp1hjQOQnMtBAqtWjeNIYKSZA1qqTP8nJGczt8H4j+Z+Px3xVOEefV6fZqe15auvIMR9rDg3PSI/dcYLmbHDzo6rXvk0yGT7DgKN4rVdL0Apgp1Q=='],
 ['NID_AUT',
  '72TP2XCZib5da7eG8p6g9PXUrwtbhxknB5uPTfAtH4/QXI20z/sTVAz4LCtxb73N'],
 ['page_uid', 'iteB2wprvmZssnKqygRssssssgd-126275'],
 ['nid_inf', '1075385671'],
 ['NNB', 'YPL7KNNXW3UWG'],
 ['nx_ssl', '2'],
 ['PM_CK_loc',
  'effb43a101d8bd43219895870fe7f4b34ce7dc3325945658bf9f7d1dcfacae4e'],
 ['SB_MODE', 'plusdeal']]

In [110]:
for kv in [l.split()[:2] for l in c.splitlines() if len(l) > 1]:
    session.cookies.set(kv[0], kv[1])

In [111]:
session.cookies

<RequestsCookieJar[Cookie(version=0, name='NID_AUT', value='72TP2XCZib5da7eG8p6g9PXUrwtbhxknB5uPTfAtH4/QXI20z/sTVAz4LCtxb73N', port=None, port_specified=False, domain='', domain_specified=False, domain_initial_dot=False, path='/', path_specified=True, secure=False, expires=None, discard=True, comment=None, comment_url=None, rest={'HttpOnly': None}, rfc2109=False), Cookie(version=0, name='NID_JKL', value='e6zeko9r9guajE5nYu5Uc5VeAFEy/LKT+MO7EyL2TEs=', port=None, port_specified=False, domain='', domain_specified=False, domain_initial_dot=False, path='/', path_specified=True, secure=False, expires=None, discard=True, comment=None, comment_url=None, rest={'HttpOnly': None}, rfc2109=False), Cookie(version=0, name='NID_SES', value='AAABqXaeyYewqNhIIphsO/IQtD9NpY2O/Ng3KUXeOThhELBgRlxGy9gbwK89AVl5Tn6hDTpTN+kOKBc4EN1MYW40oQj2BMOL61hDRsItLVBjFjrW2WExejgGF4Ek2/KFvMtQuvO9N5SrFlLAtc5QM8Wc7cd5D5PVC+Sk26OcMTibz9IXuYcgFKxMzJ+drNjKPOv4qdkuh61xBFz8MBsHMv/N4wfc/nXNFkvRo8EViaSpjoef1cSq9dkxZdFtvfiay0oq8dhU

In [112]:
resp = get('https://mail.naver.com')   # 로그인 하라고 나옴

In [113]:
print(resp.text)

<!DOCTYPE html>
<html lang="ko">
<head>
    <meta charset="UTF-8">
    <meta http-equiv="X-UA-Compatible" content="IE=edge, chrome=1">
    <meta name="viewport" content="width=device-width, initial-scale=1.0, maximum-scale=1.0, user-scalable=no">
    <meta property="og:type" content="website">
    <meta property="og:title" content="네이버">
    <meta property="og:description" content="네이버에 로그인 하고 나를 위한 다양한 서비스를 이용해 보세요">
    <meta property="og:image" content="https://ssl.pstatic.net/static/nid/account/naver_og_image.png">
    <meta property="og:image:type" content="image/png">
    <meta property="og:image:width" content="1200">
    <meta property="og:image:height" content="630">
    <title>네이버 : 로그인</title>
    <link rel="stylesheet" type="text/css" href="/login/css/global/desktop/w_20220216.css?20210812">
</head>
<body>
<div id="wrap" class="wrap">
    <div class="u_skip"><a href="https://www.naver.com">본문 바로가기</a></div>
    <header class="header" role="banner" >
        <div class="head

In [114]:
print(session.get('https://mail.naver.com').text)  # 다른 내용

<!doctype html><html><head lang="ko"><meta charset="UTF-8"/><meta name="viewport" content="width=device-width,initial-scale=1,maximum-scale=1,minimum-scale=1,viewport-fit=cover"/><title>메일</title><link rel="shortcut icon" sizes="192x192" href="https://ssl.pstatic.net/static/pwe/naver_mail/xxxhpdi_192x192.png"><link rel="apple-touch-icon-precomposed" sizes="72x72" href="https://ssl.pstatic.net/static/pwe/naver_mail/hdpi_72X72.png"><link rel="apple-touch-icon-precomposed" sizes="96x96" href="https://ssl.pstatic.net/static/pwe/naver_mail/xhdpi_96x96.png"><link rel="apple-touch-icon-precomposed" sizes="144x144" href="https://ssl.pstatic.net/static/pwe/naver_mail/xxhpdi_144x144.png"><link rel="apple-touch-icon-precomposed" sizes="192x192" href="https://ssl.pstatic.net/static/pwe/naver_mail/xxxhpdi_192x192.png"><script>window.nhn = {};
      window.gnb_option = {
        gnb_service: "mail",
        gnb_template: "gnb_utf8",
        gnb_logout: "https://www.naver.com",
        gnb_one_naver:

In [ ]:
# 로그인 하지 않고 쿠키 정보로 접근했더니 로그인 하라는 말이 안 뜨는 것을 확인할 수 있음